In [1]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


## Select Object Indices of Interest

| Object Indices |[]()|[]()|
|---|---|---|
| __1.__ 002_master_chef_can | __8.__ 009_gelatin_box      | __15.__ 035_power_drill       |
| __2.__ 003_cracker_box     | __9.__ 010_potted_meat_can  | __16.__ 036_wood_block        |
| __3.__ 004_sugar_box       | __10.__ 011_banana          | __17.__ 037_scissors          |
| __4.__ 005_tomato_soup_can | __11.__ 019_pitcher_base    | __18.__ 040_large_marker      |
| __5.__ 006_mustard_bottle  | __12.__ 021_bleach_cleanser | __19.__ 051_large_clamp       |
| __6.__ 007_tuna_fish_can   | __13.__ 024_bowl            | __20.__ 052_extra_large_clamp |
| __7.__ 008_pudding_box     | __14.__ 025_mug             | __21.__ 061_foam_brick        |

In [2]:
from transforms3d.quaternions import quat2mat, mat2quat

def getPoseCNNQuat(data, obj):
    pose_idx = np.where(data['rois'][:,1].flatten()==obj)[0]
    if(len(pose_idx) == 0):
        return None, None
    else:
        pose_idx = pose_idx[0]
    pose = data['poses'][pose_idx]
    q = pose[:4][[1,2,3,0]]
    q /= np.linalg.norm(q)
    t = pose[4:7]
    return q, t

## YCB Dataset

In [3]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes
from object_pose_utils.datasets.feature_dataset import  FeatureDataset
dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "valid"

feature_root = '/scratch/bokorn/results/posecnn_feat_all/'
feature_key = 'fc6'
feature_size = 4096

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.TRANSLATION, 
                 otypes.TRANSFORM_MATRIX,
                ]

ycb_dataset = YCBDataset(dataset_root, mode=mode,
                     object_list = object_list,
                     output_data = output_format,
                     resample_on_error = False,
                     add_syn_background = False,
                     add_syn_noise = False,
                     #use_posecnn_data = True,
                     #preprocessors = [InplaneRotator()],
                     #postprocessors = [ImageNormalizer()],
                     image_size = [640, 480], num_points=1000)

dataset = FeatureDataset(dataset_root = dataset_root,
                         feature_root = feature_root,
                         feature_key = feature_key,
                         return_pred = True,
                         mode = mode,
                         resample_on_error = False,
                         object_list = list(range(1,22)))

model_clouds = {}
for object_id in object_list:
    cloud_filename = '{}/models/{}/points.xyz'.format(dataset_root, dataset.classes[object_id])
    model_clouds[object_id] = np.loadtxt(cloud_filename)

In [4]:
df_weights = '/home/bokorn/src/DenseFusion/trained_checkpoints/ycb/pose_model_26_0.012863246640872631.pth'
df_refine_weights = '/home/bokorn/src/DenseFusion/trained_checkpoints/ycb/pose_refine_model_69_0.009449292959118935.pth'

df_results_filename = 'results_valid_df_' + '.'.join(df_weights.split('/')[-1].split('.')[:-1])
if(df_refine_weights is not None):
    df_results_filename += '_' + '.'.join(df_refine_weights.split('/')[-1].split('.')[:-1])
df_results_filename += '.pkl'

with open(df_results_filename, 'rb') as f:
    df_results = pickle.load(f)


In [5]:
ls results_valid_df_pose_model_*

results_valid_df_pose_model_13_0.02780649198161978_pose_refine_model_69_0.009449292959118935.pkl
results_valid_df_pose_model_26_0.012863246640872631_pose_refine_model_69_0.009449292959118935.pkl


## Distribution Estimators

In [6]:
from object_pose_utils.utils.pose_processing import quatAngularDiffBatch

In [7]:
from object_pose_utils.utils.interpolation import BinghamInterpolation

if(False):
    from object_pose_utils.utils.bingham import isobingham_likelihood

    sigmas_data = np.load('/home/bokorn/src/generic_pose/generic_pose/notebooks/fc6_sigma.npz', allow_pickle=True)
    sigmas = sigmas_data['sigmas'].item()

    def bing_fixed(res, obj):
        feat, pred_q, pred_t = res
        bingham = BinghamInterpolation(pred_q.unsqueeze(0).cuda(), 
                                       torch.ones(1).cuda(), 
                                       sigma=torch.Tensor([sigmas[obj.item()]]).cuda())
        return bingham, pred_q, pred_t

In [8]:
from generic_pose.models.bingham_networks import IsoBingham
from generic_pose.losses.bingham_loss import isoLikelihood

if(True):
    #feature_size = 1024
    feature_size = 4096
    #iso_model_checkpoint_pattern = '/scratch/bokorn/results/log_lik/posecnn_fc6_iso/**/weights/checkpoint_*.pth'
    #iso_model_checkpoint = sorted(glob.glob(iso_model_checkpoint_pattern,
    #                                        recursive=True))[-1]
    iso_model_checkpoint = '/scratch/bokorn/results/log_lik/posecnn_fc6_iso/lr_1e-6/2019-09-07_16-10-58/weights/best_quat.pth'
    print(glob.glob('/'.join(iso_model_checkpoint.split('/')[:-1]) + '/*', recursive=True))

    iso_estimator = IsoBingham(feature_size, len(object_list))
    iso_estimator.load_state_dict(torch.load(iso_model_checkpoint))
    iso_estimator.eval()
    iso_estimator.cuda()

    def bing_iso(res, obj):
        feat, pred_q, pred_t = res
        
        feat = torch.Tensor(feat).unsqueeze(0).cuda()
        mean_est = torch.Tensor(pred_q).unsqueeze(0).cuda()
        df_obj = torch.LongTensor(obj-1).unsqueeze(0).cuda()
        sig_est = iso_estimator(feat.unsqueeze(0).cuda(), df_obj)
        lik_est = isoLikelihood(mean_q=mean_est[0], 
                                sig=sig_est[0,0])
        
        return lik_est, pred_q, pred_t

['/scratch/bokorn/results/log_lik/posecnn_fc6_iso/lr_1e-6/2019-09-07_16-10-58/weights/best_quat.pth', '/scratch/bokorn/results/log_lik/posecnn_fc6_iso/lr_1e-6/2019-09-07_16-10-58/weights/checkpoint_524000.pth']


In [9]:
len(df_results)

41456

In [10]:
if False:
    from object_pose_utils.utils.interpolation import TetraInterpolation, BinghamInterpolation
    from generic_pose.losses.bingham_loss import isoLikelihood, duelLikelihood

    import pathlib

    from tqdm import tqdm_notebook as tqdm

    pcnn_gt = {obj:[] for obj in object_list}
    pcnn_est = {obj:[] for obj in object_list}

    df_gt = {obj:[] for obj in object_list}
    df_est = {obj:[] for obj in object_list}

    with torch.no_grad():
        for j, data in enumerate(tqdm(dataset)):
            obj, quat, trans, _ = ycb_dataset[j]
            _, feat, quat_gt, quat_pred = data
            posecnn_path = '{}/data/{}-posecnn.mat'.format(dataset.dataset_root, dataset.getPath(j))
            posecnn_data = scio.loadmat(posecnn_path)
            pred_q, pred_t = getPoseCNNQuat(posecnn_data, obj.item())
            if(len(obj) == 0):
                continue

            if(pred_q is not None):
                pcnn_gt[obj.item()].append(quat.float())
                pcnn_est[obj.item()].append(torch.Tensor(pred_q))

            if(j in df_results.keys()):
                res = df_results[j]
                if(res is not None):
                    df_gt[obj.item()].append(quat.float())
                    df_est[obj.item()].append(torch.Tensor(res['max_q']).float())

    for obj in object_list:
        df_gt[obj] = torch.stack(df_gt[obj])
        df_est[obj] = torch.stack(df_est[obj])
        pcnn_gt[obj] = torch.stack(pcnn_gt[obj])
        pcnn_est[obj] = torch.stack(pcnn_est[obj])
    
    np.savez('val_quats.npz', 
         df_gt=df_gt,
         df_est=df_est,
         pcnn_gt=pcnn_gt,
         pcnn_est=pcnn_est)

Exception on index 11695: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0090/000617_007_tuna_fish_can_feat.npz'
Exception on index 19642: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0087/000214_010_potted_meat_can_feat.npz'
Exception on index 19643: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0087/000215_010_potted_meat_can_feat.npz'
Exception on index 19644: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0087/000216_010_potted_meat_can_feat.npz'
Exception on index 19645: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0087/000217_010_potted_meat_can_feat.npz'
Exception on index 19647: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0087/000219_010_potted_meat_can_feat.npz'
Exception on index 19649: [Errno 2] No such file or directory: '/scratch/bokorn/results/po

In [13]:
from generic_pose.losses.bingham_loss import isoLoss
q_gt = df_gt[1].cuda()
q_df = df_est[1].cuda()

In [14]:
for sig = 
    sig = torch.Tensor([[10]]).repeat(q_gt.shape[0], 1).cuda()
isoLoss(q_df, sig, q_gt)

(tensor(-0.6448, device='cuda:0'), tensor(2.0323, device='cuda:0'))

In [15]:
q_gt

tensor([[ 0.3285,  0.8043,  0.3770,  0.3211],
        [ 0.3299,  0.8038,  0.3766,  0.3214],
        [ 0.3279,  0.8050,  0.3759,  0.3213],
        ...,
        [-0.6503,  0.6982, -0.2303, -0.1913],
        [-0.6499,  0.6994, -0.2292, -0.1898],
        [-0.6509,  0.6990, -0.2283, -0.1884]], device='cuda:0')

0.8719625980246123